# Домашнее задание №9

**Задание 1**

Напишите функцию, которая классифицирует фильмы из материалов занятия по правилам:
- оценка 2 и ниже — низкий рейтинг;
- оценка 4 и ниже — средний рейтинг;
- оценка 4.5 и 5 — высокий рейтинг.

Результат классификации запишите в столбец class.

In [1]:
#импорт необходимых библиотек
import pandas as pd

In [2]:
#загрузка данных из CSV файлов
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')

In [3]:
#знакомство с данными
print("Первые 5 строк таблицы movies:")
print(movies_df.head())


print("\nПервые 5 строк таблицы ratings:")
print(ratings_df.head())

Первые 5 строк таблицы movies:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  

Первые 5 строк таблицы ratings:
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [4]:
#напишем функцию для классификации рейтингов
def classify_rating(rating):
    if rating <= 2:
        return 'низкий рейтинг'
    elif rating <= 4:
        return 'средний рейтинг'
    elif rating <= 5:
        return 'высокий рейтинг'
    else:
        return 'рейтинг не определен'

In [5]:
#применим написанную функцию к столбцу ratings 
#и добавим новый столбец в нашу таблицу
ratings_df['class'] = ratings_df['rating'].apply(classify_rating)
ratings_df.head()

,userId,movieId,rating,timestamp,class
0,1,1,4.0,964982703,средний рейтинг
1,1,3,4.0,964981247,средний рейтинг
2,1,6,4.0,964982224,средний рейтинг
3,1,47,5.0,964983815,высокий рейтинг
4,1,50,5.0,964982931,высокий рейтинг


In [6]:
#выведем df с полной ифнормации по фильмам
movies_with_rating = ratings_df.merge(movies_df, on='movieId', how='left')
movies_with_rating.head()

,userId,movieId,rating,timestamp,class,title,genres
0,1,1,4.0,964982703,средний рейтинг,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,средний рейтинг,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,средний рейтинг,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,высокий рейтинг,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,высокий рейтинг,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


**Задание 2**

Примечание: *Используйте файл keywords.csv.*

Нужно написать гео-классификатор, который каждой строке сможет выставить географическую при надлежность определённому региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

**Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:**

geo_data = {
    
    'Центр': ['москва', 'тула', 'ярославль'],
    
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [7]:
#импорт необходимых библиотек
#import pandas as pd #импорт был произведен ранее

In [8]:
#загрузка данных из CSV файлов
keywords_df = pd.read_csv('keywords.csv')

In [9]:
#знакомство с данными
keywords_df.head(20)

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195
5,одноклассники моя страница,19976138
6,майл,15144673
7,авито,14786593
8,переводчик,14453469
9,яндекс,13344734


In [10]:
#словарь, который хранит информацию о географических регионах
geo_data = {
'Центр': ['белгород','брянск', 'владимир', 'воронеж', 'иваново', 'калуга', 'москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск','карелия','коми', 'архангельск', 'вологда', 'калининград'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск','бурятия', 'саха','якутия','магадан', 'чукотка']
}

In [11]:
# создадим геокласссификатор, который будет определять принадлежность запроса к определенной территории
def classify_region(query):
    query = query.lower()  #приведем запрос к нижнему регистру, 
                           #чтобы исключить "чувствительность" нашего классификатора к регистру
    for region, cities in geo_data.items():
        for city in cities:
            if city in query:
                return region  #возвращаем название региона, если упомянутый в запросе город
                               #был найден в словаре
    return 'регион не определен' 

In [12]:
#применим нашу функцию и добавим новый столбец в df
keywords_df['region'] = keywords_df['keyword'].apply(classify_region)

In [13]:
keywords_df = keywords_df[keywords_df['region'] != 'регион не определен']#отфильтруем наш df
keywords_df.head(20)

,keyword,shows,region
127,авито москва,979292,Центр
230,порно комиксы,629484,Северо-Запад
370,авито ру санкт петербург,425134,Северо-Запад
563,авито воронеж,294335,Центр
564,погода в санкт петербурге,292489,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
1092,авито белгород,173992,Центр
1115,авито брянск,171239,Центр
1185,путин владимир владимирович,164596,Центр
